# Data Understanding and Cleaning

The aim of the following notebook is to understand the structure and identify potencial data quality issues that could affect for further exploration data analysis and RFM calculations.

The notebook is divided in the following sub-sections:
- Dataset dimensions
- Dataset colum data-type and meaning
- Missing values
- Column's value ranges
- Duplicate values

First it's necessary to import the Python libraries needed for this initial data exploration. In the same cell, the path of the file and the method used to read it are included.

In [2]:
import pandas as pd

path = "../data_raw/online_retail_II.csv"
df = pd.read_csv(path)

## Dataset dimensions
According to the result shown below, there is a rectangular dataset with 1,067,371 instances of height and 8 columns of width.

In [4]:
df.shape

(1067371, 8)

## Dataset colum data-type and meaning
The dataset is compound of the following 8 columns:
| Column | Data Type | Description |
|---|---|---|
| Invoice | Nominal, String | A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. |
| StockCode | Nominal, String | A 5-digit integral number uniquely assigned to each distinct product. |
| Description | Nominal, String | Product item name. |
| Quantity | Numeric, Int64 | The quantities of each product (item) per transaction. |
| InvoiceDate | Numeric, String | The day and time when a transaction was generated. |
| Price | Numeric, Float64 | Product price per unit in sterling |
| Customer ID | Nominal, Float64 | A 5-digit integral number uniquely assigned to each customer. |
| Country | NOminal, String | The name of the country where a customer resides. |

## Missing values
However, not all the attributes above have their entire registries with not null values, as we can see in the cell result below. Both _Description_ and _Customer ID_ have some rows with null values, being the last one the column with the highest count of null values.

Due to the main goal of the project, which is to implement a RFM analysis, the _Description_ column will be discharged because its lack of relevance for this specific type of analysis. Moreover, _Customer ID_'s null values will be removed in the cleaning process, including the rest of the attributes of the corresponding rows, since RFM analysis requires customer level identification. This decision reduces the total number of records but ensures accurate and meaningful customer segmentation.

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  str    
 1   StockCode    1067371 non-null  str    
 2   Description  1062989 non-null  str    
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  str    
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 65.1 MB


### Cleaning Customer ID missing values
Before continue with the rest of the data understanding process, transactions without a valid _Customer ID_ value will be removed. This is to avoid hindering the analysis of the remaining data.

In [9]:
df_v1 = df.dropna(subset=['Customer ID'])

In [10]:
df_v1.info()

<class 'pandas.DataFrame'>
Index: 824364 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      824364 non-null  str    
 1   StockCode    824364 non-null  str    
 2   Description  824364 non-null  str    
 3   Quantity     824364 non-null  int64  
 4   InvoiceDate  824364 non-null  str    
 5   Price        824364 non-null  float64
 6   Customer ID  824364 non-null  float64
 7   Country      824364 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 56.6 MB


## Column's value ranges
For the numerical values, it is important to know what the extreme value ranges are and if they are accurate according to the business logic. Therefore, after calling the describe function, we found the minimum value for _Quantity_ was negative, which could represent devolutions or cancellations; additionally, the lowest value of the _Price_ column is zero, which doesn't represent a real monetary transaction.

Overall, _Quantity_ negative values and _Price_ zero values must be excluded from the analysis to prevent distorsions in the recency, frequency and monetary values of the customers.

In [ ]:
df_v1.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


In [17]:
quantity_sorted_df = df_v1.sort_values(by="Quantity")
quantity_sorted_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1065883,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
529729,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277.0,France


In [18]:
price_sorted_df = df_v1.sort_values(by="Price")
price_sorted_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
248583,513416,22423,REGENCY CAKESTAND 3 TIER,5,2010-06-24 12:34:00,0.0,13089.0,United Kingdom
392008,527084,22630,DOLLY GIRL LUNCH BOX,64,2010-10-14 15:33:00,0.0,14646.0,Netherlands
16107,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
612250,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
945865,572893,21208,PASTEL COLOUR HONEYCOMB FAN,5,2011-10-26 14:36:00,0.0,18059.0,United Kingdom


### Cleaning Quantity and Price minimum values
Before continue with the rest of the data understanding process, transactions with a negative quantity or a zero price will be removed. This is to avoid hindering the analysis of the remaining data.

In [20]:
df_v2 = df_v1[(df_v1["Quantity"] > 0) & (df_v1["Price"] > 0)]

In [21]:
df_v2.info()

<class 'pandas.DataFrame'>
Index: 805549 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      805549 non-null  str    
 1   StockCode    805549 non-null  str    
 2   Description  805549 non-null  str    
 3   Quantity     805549 non-null  int64  
 4   InvoiceDate  805549 non-null  str    
 5   Price        805549 non-null  float64
 6   Customer ID  805549 non-null  float64
 7   Country      805549 non-null  str    
dtypes: float64(2), int64(1), str(5)
memory usage: 55.3 MB
